In [ ]:
# importing libraries 
!pip install seaborn 
!pip install pickle5
!pip install torch torchvision
!pip install transformers
!pip install evaluate
!pip install rouge_score
!pip install pysbd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 28.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 29.4 MB/s 
     |████████████████████████████████| 163 kB 60.0 MB/s 
     |████████████████████████████████| 7.6 MB 54.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.3 MB/s 
     |████████████████████████████████| 441 kB 45.3 MB/s 
     |████████████████████████████████| 212 kB 54.3 MB/s 
     |████████████████████████████████| 115 kB 71.9 MB/s 
     |████████████████████████████████| 95 

In [ ]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

import pickle5 as pickle
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import evaluate

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_colwidth', 30)

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
## python 'your_file.py' 'task_name' : baseline
## python 'your_file.py' 'task_name' --finetune : finetuning  

import os 
import argparse
import re
from bs4 import BeautifulSoup

## finetune lib
import transformers
import torch
import nltk
nltk.download('punkt')
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset

## holding lib
from nltk.tokenize import sent_tokenize
import gc

# cleaning html tags
def clean_html(raw_html):
    cleantext = BeautifulSoup(raw_html, "html.parser").text
    cleantext = re.sub('\xa0', ' ', cleantext)
    cleantext = re.sub('\n', ' ', cleantext)
    return cleantext

def generate_task(file_path):
    
    print('1. Generating data...')
    # open file 
    oyez = pickle.load(open(file_path, "rb"))
    
    # drop duplicates
    oyez = oyez.drop_duplicates()
    
    # creating some conditions to check non-null counts
    input = ~oyez['Justia_txt'].isnull()
    task1 = ~oyez['Justia_summary'].isnull()
    
    # generating dataset for task1
    task_df = oyez[input&task1]
    
    # Data cleaning 
    # all remove unknown answers
    task = 'Justia_summary'
    task_df[task] = task_df[task].apply(lambda x: clean_html(x))

    current_unk = ['Annotation', 'Currently unknown.', 'Currently unavailable.', '']
    task_df = task_df[~task_df[task].isin(current_unk)]
    task_df = task_df.reset_index().drop(columns=['index'])

    task_df = task_df[['file_name_oyez', 'Justia_txt', task]]
    task_df.columns = ['id', 'input', 'target']

    print('Done!')
    print('Shape of dataset: ', task_df.shape)
    print()
    
    return task_df        

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
file_path = 'gdrive/My Drive/DSGA1006_worldbank/codes/data/NYU_Oyez_data.pkl'
task_df = generate_task(file_path)
task_df

1. Generating data...
Done!
Shape of dataset:  (781, 3)



,id,input,target
0,2021_21-429,"NOTE:Where it is feasible,...",Castro-Huerta was convicte...
1,2021_21-954,"NOTE:Where it is feasible,...","In 2019, the Department of..."
2,2021_21-511,"NOTE:Where it is feasible,...",Twyford was convicted of a...
3,2021_21-439,"NOTE:Where it is feasible,...",A prisoner who challenges ...
4,2021_21-499,"NOTE:Where it is feasible,...",Los Angeles County Deputy ...
...,...,...,...
776,2010_09-993,"PLIVA, INC., etal. v. MENS...",These consolidated lawsuit...
777,2010_10-188,SCHINDLER ELEVATOR CORP. v...,Respondent filed an action...
778,2010_10-5400,TAPIA v. UNITED STATES cer...,Petitioner was convicted o...
779,2010_09-1227,BOND v. UNITED STATES cert...,Petitioner was indicted fo...


In [ ]:
# here we divide train, test dataset first 
train_df = task_df.sample(frac=0.8,random_state=0).reset_index().drop(columns='index')
test_df = task_df.drop(train_df.index).reset_index().drop(columns='index')

In [ ]:
train_df.shape, test_df.shape

((625, 3), (156, 3))

In [ ]:
# divide into list of sentences from entire doc by using pysbd
# this applies to train/test input, and train target set 
import spacy
from pysbd.utils import PySBDFactory

# creating a spacy variable 
nlp = spacy.blank('en')
nlp.add_pipe('sentencizer')

# this converts a paragraph into list of sentences 
train_df['input_pysbd'] = train_df['input'].apply(lambda x: list(nlp(x).sents))
train_df['target_pysbd'] = train_df['target'].apply(lambda x: list(nlp(x).sents))

In [ ]:
# cos sim 
from numpy.linalg import norm

def cos_sim(ref_sent, sent):

    # to compare, we convert each sentences into list 
    # note that as sentences will be sent to torch, 
    # we convert into numpy 
    list_a = ref_sent.detach().cpu().numpy()
    list_b = sent.detach().cpu().numpy()

    # calculate based on the cosine similarity formula 
    cos_sim_score = np.dot(list_a,list_b)/((norm(list_a)*norm(list_b)))
    
    # return cosine similarity score 
    return cos_sim_score

# algo 2
def semantic_sim(sentence, curr_chunk, next_chunk):
    
    # initialize cos sim scores for both curr and next chunk 
    score_curr_chunk = 0
    score_next_chunk = 0
    
    # for each sentences in current chunk, add cos_sim score
    for enc_chunk_sentence in curr_chunk: 
      score_curr_chunk += cos_sim(enc_chunk_sentence, sentence)

    # also, for each sentences in next chunk, add cos_sim score
    for enc_chunk_sentence in next_chunk: 
      score_next_chunk += cos_sim(enc_chunk_sentence, sentence)

    # then we divide into each size of chunk to get an average score 
    score_curr_chunk /= len(curr_chunk)
    score_next_chunk /= len(next_chunk)  

    # compare the score, and append to the chunk where the score is higher 
    if score_curr_chunk > score_next_chunk:
      curr_chunk.append(sentence)
    else:
      next_chunk.append(sentence)

# algo 3
# objective of this function is to create a target chunk and distribute sentences based on main_chunk
def target_assign(summary_sentences, main_chunk):
  
  # initialize targets, rouge score
  targets = [[] for i in range(len(main_chunk))]
  rouge = evaluate.load('rouge')

  # as all documents would not have string format of the sentence, we change sentences into string format
  # i.e. word -> 'word' 
  summary_sentences = [str(input) for input in summary_sentences]  
  
  # then for each target sentences, check rouge score for each of main chunk
  for sentence in summary_sentences:
    rouge_scores = []
    
    # for each chunk, we calculate rouge score and append to the list
    for c in main_chunk:
      
      chunk_score = rouge.compute(predictions=[c], references=[sentence])
      rouge_scores.append(chunk_score['rougeL']) # rougeL, this can be changed to any evaluation metric

    # then we select the highest rouge score index and append to the target chunk index 
    idx = np.argmax(rouge_scores)
    targets[idx].append(sentence)

  # after that, we merge list of sentences to one paragraph 
  for i in range(len(targets)):
    targets[i] = " ".join(str(label) for label in targets[i])

  # return target chunks 
  return targets      

In [ ]:
# algo1
from transformers import AutoTokenizer, AutoModel
import numpy as np
from numpy.linalg import norm

## algo 1: Note that we seperate algo 3 in this function
# this happens as we don't know test set 
def create_chunk(document, lower_size, upper_size, target=None):
  # assigning chunks 
  main_chunk = []
  curr_chunk = []
  next_chunk = []

  # also setting lower / upper size
  lower_size = lower_size
  upper_size = upper_size
  tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")

  # To compare similarity score, we encode document 
  doc_str_converted = [str(input) for input in document]  
  enc_doc_str_converted = tokenizer(doc_str_converted, max_length=tokenizer.model_max_length, padding="longest", truncation=True, return_tensors='pt')

  # main function
  for enc_sentence in enc_doc_str_converted['input_ids']:
      # print(enc_sentence)
      # fill min sentences for current chunk, and next chunk 
      if len(curr_chunk) + 1 < lower_size:
          curr_chunk.append(enc_sentence)
      elif len(next_chunk) + 1 < lower_size:
          next_chunk.append(enc_sentence)

      # if any of the chunks are filled, move to main chunk
      elif len(curr_chunk) + 1 > upper_size:
          main_chunk.append(curr_chunk)
          curr_chunk = []
      elif len(next_chunk) + 1 > upper_size:
          main_chunk.append(next_chunk)
          next_chunk = []        

      # for rest of the sentences, conduct similarity comparison and append sentence to 
      # either current or next chunk
      else:
          semantic_sim(enc_sentence, curr_chunk, next_chunk)

  # safety code: this prevents the case where curr_chunk and next_chunk are piled up but not added to the main chunk 
  # because they didn't reached the upper size  
  if len(curr_chunk) != 0:
    main_chunk.append(curr_chunk)
  if len(next_chunk) != 0:
    main_chunk.append(next_chunk)

  ## this makes decoded chunks of main_chunk 
  decoded_main_chunk = [tokenizer.batch_decode(chunk, skip_special_tokens=True) for chunk in main_chunk] 

  ## converting seperated sentences into chunk 
  for i in range(len(decoded_main_chunk)):
    decoded_main_chunk[i] = " ".join(str(label) for label in decoded_main_chunk[i])

  ## this creates either input chunk or (input,output) pair of chunks 
  # this is generating only chunk
  if target == None:
    return decoded_main_chunk

  # this is generating (input,output) pair of chunks 
  else:

    # this gives a list of target sentences distributed to target chunks 
    target_chunks = target_assign(target, decoded_main_chunk)

    ## this only returns decodecd chunk and target chunk pair that exists
    filtered_main_chunk = []
    filtered_target_chunk = []

    for i in range(len(target_chunks)):
      if target_chunks[i] != '':
        filtered_main_chunk.append(decoded_main_chunk[i])
        filtered_target_chunk.append(target_chunks[i])

    # return filtered (input, target) chunk pairs 
    return filtered_main_chunk, filtered_target_chunk

In [ ]:
# this is the main function to run
input_chunks_list = []
target_chunks_list = []

# for train set, we iterate every doc-summary pair and create chunk-target pair 
for i in range(len(train_df)):
  input_chunks, target_chunks = create_chunk(train_df['input_pysbd'][i], lower_size=10, upper_size=50, target = train_df['target_pysbd'][i])
  
  input_chunks_list.extend(input_chunks)
  target_chunks_list.extend(target_chunks)

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/222k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# then we convert into dataframe 
df = pd.DataFrame(list(zip(input_chunks_list, target_chunks_list)), columns =['input', 'target'])
df

In [ ]:
# # save checkpoint
# file_name = 'gdrive/My Drive/DSGA1006_worldbank/codes/preprocess_result_1115_10_50.csv' # your path
# df.to_csv(file_name, index=False)

In [ ]:
# ## load df
# file_name = 'gdrive/My Drive/DSGA1006_worldbank/codes/preprocess_result_1115.csv'
# df = pd.read_csv(file_name)
# df.columns = ['input', 'target']

In [ ]:
# preprocssing. this encodes input (and labels)  
def preprocess_function(df):

    model_inputs = tokenizer(df["input"], max_length=tokenizer.model_max_length, truncation=True)

    # Setup the tokenizer for targets
    # with tokenizer.as_target_tokenizer():
    labels = tokenizer(df["target"], max_length=tokenizer.model_max_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# training function, returns the model (which is trainer)
def model_train(train_df, test_df):
    # define args for finetuning
    batch_size = 2
    args = Seq2SeqTrainingArguments(
        f"{model_name}-wb-finetuned-1114",
        evaluation_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=20, # or 20
        predict_with_generate=True,
        # fp16=True,
        # push_to_hub=True,
    )

    # collator fixed of max length 
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    trainer = Seq2SeqTrainer(
        model,
        args,
        train_dataset=train_df,
        eval_dataset=test_df,
        data_collator=data_collator,
        tokenizer=tokenizer,
    )

    trainer.train()

    return trainer    

# decoding function, this converts list of intergers into sentence 
def decode_data(prediction, label):
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(label_ids != -100, label_ids, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # We convert back into the sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    return decoded_preds, decoded_labels

In [ ]:
## In case of Finetuning 
# convert df to Dataset(huggingface) class 

# define your model
model_name = 'facebook/bart-base'  # bart-base
tokenizer_name = 'facebook/bart-base' # bart-base
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

dataset = Dataset.from_pandas(df)
train_test_df = dataset.train_test_split(test_size=0.2)

# preprocess dataset
tokenized_datasets = train_test_df.map(preprocess_function, batched=True)

# finetune the model
train = tokenized_datasets["train"]
test = tokenized_datasets["test"]

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer = model_train(train, test)

In [ ]:
# preprocssing. this encodes input (and labels)  
def preprocess_function_testset(df):
    # we only encode input 
    model_inputs = tokenizer(df["input"], max_length=tokenizer.model_max_length, truncation=True)        
    return model_inputs

# decoding function, this converts list of intergers into sentence 
def decode_data_testset(prediction):
    decoded_preds = tokenizer.batch_decode(prediction, skip_special_tokens=True)
    
    # We convert back into the sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]

    return decoded_preds    

# evaluate by rouge score, we focus on rougeL
def evaluate_result(df):
    print('3. Evalating...')
    rouge = evaluate.load('rouge')
    result = rouge.compute(predictions=df['pred'].values.tolist(), references=df['target'].values.tolist())
    
    print('Done!')
    return result    

In [ ]:
# divide into list of sentences from entire doc by using pysbd
# this applies to train/test input, and train target set 
import spacy
from pysbd.utils import PySBDFactory

nlp = spacy.blank('en')
nlp.add_pipe('sentencizer')

test_df['input_pysbd'] = test_df['input'].apply(lambda x: list(nlp(x).sents))

In [ ]:
decoded_preds_merged_list = []
for i in range(len(test_df)):

  input_chunks_list = []
  input_chunks = create_chunk(test_df['input_pysbd'][i], lower_size=10, upper_size=50)
  
  input_chunks_list.extend(input_chunks)

  df = pd.DataFrame(input_chunks_list, columns = ['input'])
  dataset = Dataset.from_pandas(df)

  # preprocess dataset
  tokenized_datasets = dataset.map(preprocess_function_testset, batched=True)

  # predict: if 10 chunks -> 10 summary 
  predictions = trainer.predict(tokenized_datasets, max_length=tokenizer.model_max_length)

  # decode prediction result 
  decoded_preds = decode_data_testset(predictions[0])

  decoded_preds_merged =  " ".join(decoded_preds)

  decoded_preds_merged_list.append(decoded_preds_merged)

In [ ]:
# convert dataset to dataframe and add prediction column 
test_df['pred'] = decoded_preds_merged_list

In [ ]:
# evaluate and print the result
result = evaluate_result(test_df)
print(result)
print()

In [ ]:
## check some examples 
print('4. Examples: ')
for i in range(5):
    print('case:', i) 
    print('pred length:', len(test_df.loc[i, 'pred']))
    print('prediction: ', test_df.loc[i, 'pred'])
    print()
    print('actual length:', len(test_df.loc[i, 'target']))
    print('actual: ', test_df.loc[i, 'target'])
    print()

In [ ]:
# save checkpoint
file_name = 'gdrive/My Drive/DSGA1006_worldbank/codes/result_1115_se3_yoon.csv'
test_df.to_csv(file_name, index=False)